<img src="/home/ahmedunix/data_Science_Work/Machine_Learning/18_Ensemble_Learning_Bagging/excercise/bagging_excercise.png">

In [1]:
import math
import kaggle as kaggle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import tensorflow as tf, keras
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

2024-10-07 14:26:20.978099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-07 14:26:21.277162: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-07 14:26:21.358819: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-07 14:26:25.309469: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [13]:
!kaggle datasets download -d fedesoriano/heart-failure-prediction

Dataset URL: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction
License(s): ODbL-1.0
  0%|                                               | 0.00/8.56k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 8.56k/8.56k [00:00<00:00, 12.3MB/s]


In [14]:
!unzip heart-failure-prediction.zip -d heart_failure_prediction_dataset

Archive:  heart-failure-prediction.zip
  inflating: heart_failure_prediction_dataset/heart.csv  


In [2]:
df = pd.read_csv("/home/ahmedunix/data_Science_Work/Machine_Learning/18_Ensemble_Learning_Bagging/excercise/heart_failure_prediction_dataset/heart.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [4]:
df.isna().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [5]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [6]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [7]:
df.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0


In [8]:
df['age_z_score'] = (df['Age'] - df['Age'].mean()) / df['Age'].std()
df['RestingBP_z_score'] = (df['RestingBP'] - df['RestingBP'].mean()) / df['RestingBP'].std()
df['Cholesterol_z_score'] = (df['Cholesterol'] - df['Cholesterol'].mean()) / df['Cholesterol'].std()
df['MaxHR_z_score'] =(df['MaxHR'] - df['MaxHR'].mean()) / df['MaxHR'].std()
df['Oldpeak_z_score'] = (df['Oldpeak'] - df['Oldpeak'].mean()) / df['Oldpeak'].std()

In [9]:
df.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,age_z_score,RestingBP_z_score,Cholesterol_z_score,MaxHR_z_score,Oldpeak_z_score
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0,-1.432359,0.410685,0.824621,1.382175,-0.831979
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1,-0.478223,1.490940,-0.171867,0.753746,0.105606
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0,-1.750404,-0.129442,0.769768,-1.524307,-0.831979
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1,-0.584238,0.302660,0.138964,-1.131539,0.574398
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0,0.051853,0.950812,-0.034736,-0.581664,-0.831979
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0,-1.538374,-0.669570,1.281725,1.303621,-0.831979
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0,-0.902283,-0.129442,0.349232,1.303621,-0.831979
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0,0.051853,-1.209697,0.084111,0.203871,-0.831979
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1,-1.750404,0.410685,0.074969,-0.267450,0.574398
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0,-0.584238,-0.669570,0.778910,-0.660218,-0.831979


In [10]:
df['Outlier'] = ((df['age_z_score'] >= 3) | (df['RestingBP_z_score'] >= 3)
                 | (df['Cholesterol_z_score'] >= 3) | (df['MaxHR_z_score'] >= 3)
                 | (df['Oldpeak_z_score'] >= 3) | (df['age_z_score'] <= -3)
                 | (df['RestingBP_z_score'] <= -3) | (df['Cholesterol_z_score'] <= -3)
                 | (df['MaxHR_z_score'] <= -3) | (df['Oldpeak_z_score'] <= -3)
)

In [11]:
df.head(12)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,age_z_score,RestingBP_z_score,Cholesterol_z_score,MaxHR_z_score,Oldpeak_z_score,Outlier
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0,-1.432359,0.410685,0.824621,1.382175,-0.831979,False
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1,-0.478223,1.490940,-0.171867,0.753746,0.105606,False
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0,-1.750404,-0.129442,0.769768,-1.524307,-0.831979,False
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1,-0.584238,0.302660,0.138964,-1.131539,0.574398,False
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0,0.051853,0.950812,-0.034736,-0.581664,-0.831979,False
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0,-1.538374,-0.669570,1.281725,1.303621,-0.831979,False
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0,-0.902283,-0.129442,0.349232,1.303621,-0.831979,False
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0,0.051853,-1.209697,0.084111,0.203871,-0.831979,False
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1,-1.750404,0.410685,0.074969,-0.267450,0.574398,False
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0,-0.584238,-0.669570,0.778910,-0.660218,-0.831979,False


In [12]:
df_no_outlier = df[df['Outlier']==False]

In [13]:
df_no_outlier

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,age_z_score,RestingBP_z_score,Cholesterol_z_score,MaxHR_z_score,Oldpeak_z_score,Outlier
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0,-1.432359,0.410685,0.824621,1.382175,-0.831979,False
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1,-0.478223,1.490940,-0.171867,0.753746,0.105606,False
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0,-1.750404,-0.129442,0.769768,-1.524307,-0.831979,False
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1,-0.584238,0.302660,0.138964,-1.131539,0.574398,False
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0,0.051853,0.950812,-0.034736,-0.581664,-0.831979,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1,-0.902283,-1.209697,0.596068,-0.188897,0.293123,False
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1,1.536064,0.626736,-0.053020,0.164595,2.355810,False
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1,0.369898,-0.129442,-0.619830,-0.856602,0.293123,False
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1,0.369898,-0.129442,0.340090,1.460728,-0.831979,False


In [31]:
resting_ecg = df_no_outlier['RestingECG'].unique()
resting_ecg

array(['Normal', 'ST', 'LVH'], dtype=object)

In [32]:
oe_resting_ecg = OrdinalEncoder(categories=[resting_ecg])

In [33]:
df_no_outlier['RestingECG'] = oe_resting_ecg.fit_transform(df_no_outlier[['RestingECG']])

/tmp/ipykernel_4814/3493191262.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_outlier['RestingECG'] = oe_resting_ecg.fit_transform(df_no_outlier[['RestingECG']])


In [27]:
exercise_Angina = df_no_outlier['ExerciseAngina'].unique()
exercise_Angina

array(['N', 'Y'], dtype=object)

In [28]:
oe_excercise_engina = OrdinalEncoder(categories=[exercise_Angina])

In [29]:
df_no_outlier['ExerciseAngina'] = oe_excercise_engina.fit_transform(df_no_outlier[['ExerciseAngina']])

/tmp/ipykernel_4814/3347777547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_outlier['ExerciseAngina'] = oe_excercise_engina.fit_transform(df_no_outlier[['ExerciseAngina']])


In [39]:
st_slope = df_no_outlier['ST_Slope'].unique() 
st_slope_list = ['Down','Up', 'Flat']

In [40]:
oe_st_slope = OrdinalEncoder(categories=[st_slope_list])

In [41]:
df_no_outlier['ST_Slope'] = oe_st_slope.fit_transform(df_no_outlier[['ST_Slope']])

/tmp/ipykernel_4814/1716320840.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_outlier['ST_Slope'] = oe_st_slope.fit_transform(df_no_outlier[['ST_Slope']])


In [42]:
df_no_outlier

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,age_z_score,RestingBP_z_score,Cholesterol_z_score,MaxHR_z_score,Oldpeak_z_score,Outlier
0,40,M,ATA,140,289,0,0.0,172,0.0,0.0,1.0,0,-1.432359,0.410685,0.824621,1.382175,-0.831979,False
1,49,F,NAP,160,180,0,0.0,156,0.0,1.0,2.0,1,-0.478223,1.490940,-0.171867,0.753746,0.105606,False
2,37,M,ATA,130,283,0,1.0,98,0.0,0.0,1.0,0,-1.750404,-0.129442,0.769768,-1.524307,-0.831979,False
3,48,F,ASY,138,214,0,0.0,108,1.0,1.5,2.0,1,-0.584238,0.302660,0.138964,-1.131539,0.574398,False
4,54,M,NAP,150,195,0,0.0,122,0.0,0.0,1.0,0,0.051853,0.950812,-0.034736,-0.581664,-0.831979,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,0.0,132,0.0,1.2,2.0,1,-0.902283,-1.209697,0.596068,-0.188897,0.293123,False
914,68,M,ASY,144,193,1,0.0,141,0.0,3.4,2.0,1,1.536064,0.626736,-0.053020,0.164595,2.355810,False
915,57,M,ASY,130,131,0,0.0,115,1.0,1.2,2.0,1,0.369898,-0.129442,-0.619830,-0.856602,0.293123,False
916,57,F,ATA,130,236,0,2.0,174,0.0,0.0,2.0,1,0.369898,-0.129442,0.340090,1.460728,-0.831979,False


In [43]:
ohe = OneHotEncoder(handle_unknown='ignore',sparse_output=False).set_output(transform='pandas')

In [44]:
ohe_transform = ohe.fit_transform(df_no_outlier[['Sex','ChestPainType']])

In [45]:
ohe_transform

,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,0.0,1.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0
3,1.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
913,0.0,1.0,0.0,0.0,0.0,1.0
914,0.0,1.0,1.0,0.0,0.0,0.0
915,0.0,1.0,1.0,0.0,0.0,0.0
916,1.0,0.0,0.0,1.0,0.0,0.0


In [46]:
ohe_df = pd.concat([df_no_outlier,ohe_transform],axis='columns').drop(
    columns=['Sex','Sex_F','ChestPainType','ChestPainType_TA',
             'age_z_score','RestingBP_z_score','Cholesterol_z_score',
             'MaxHR_z_score','Oldpeak_z_score','Outlier']
)

In [47]:
ohe_df

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP
0,40,140,289,0,0.0,172,0.0,0.0,1.0,0,1.0,0.0,1.0,0.0
1,49,160,180,0,0.0,156,0.0,1.0,2.0,1,0.0,0.0,0.0,1.0
2,37,130,283,0,1.0,98,0.0,0.0,1.0,0,1.0,0.0,1.0,0.0
3,48,138,214,0,0.0,108,1.0,1.5,2.0,1,0.0,1.0,0.0,0.0
4,54,150,195,0,0.0,122,0.0,0.0,1.0,0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,110,264,0,0.0,132,0.0,1.2,2.0,1,1.0,0.0,0.0,0.0
914,68,144,193,1,0.0,141,0.0,3.4,2.0,1,1.0,1.0,0.0,0.0
915,57,130,131,0,0.0,115,1.0,1.2,2.0,1,1.0,1.0,0.0,0.0
916,57,130,236,0,2.0,174,0.0,0.0,2.0,1,0.0,0.0,1.0,0.0


In [48]:
x = ohe_df.drop(columns=['HeartDisease'])
y = ohe_df.HeartDisease

In [49]:
scaler = StandardScaler().set_output(transform='pandas')

In [50]:
x_scaled = scaler.fit_transform(x)

In [52]:
x_scaled

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP
0,-1.428154,0.465900,0.849636,-0.550362,-0.745031,1.384320,-0.822945,-0.855469,-0.714228,0.515943,-1.077524,2.063325,-0.534905
1,-0.475855,1.634714,-0.168122,-0.550362,-0.745031,0.752973,-0.822945,0.137516,0.923759,-1.938199,-1.077524,-0.484655,1.869492
2,-1.745588,-0.118507,0.793612,-0.550362,0.499918,-1.535661,-0.822945,-0.855469,-0.714228,0.515943,-1.077524,2.063325,-0.534905
3,-0.581666,0.349019,0.149344,-0.550362,-0.745031,-1.141069,1.215148,0.634008,0.923759,-1.938199,0.928054,-0.484655,-0.534905
4,0.053200,1.050307,-0.028064,-0.550362,-0.745031,-0.588640,-0.822945,-0.855469,-0.714228,0.515943,-1.077524,-0.484655,1.869492
...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,-0.899099,-1.287320,0.616205,-0.550362,-0.745031,-0.194048,-0.822945,0.336112,0.923759,0.515943,-1.077524,-0.484655,-0.534905
914,1.534554,0.699663,-0.046738,1.816985,-0.745031,0.161085,-0.822945,2.520678,0.923759,0.515943,0.928054,-0.484655,-0.534905
915,0.370633,-0.118507,-0.625646,-0.550362,-0.745031,-0.864854,1.215148,0.336112,0.923759,0.515943,0.928054,-0.484655,-0.534905
916,0.370633,-0.118507,0.354763,-0.550362,1.744868,1.463238,-0.822945,-0.855469,0.923759,-1.938199,-1.077524,2.063325,-0.534905


In [53]:
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,stratify=y,random_state=20,test_size=0.2)

In [56]:
model_svm = SVC()
model_svm.fit(x_train,y_train)

SVC()

In [57]:
model_svm.score(x_test,y_test)

0.85

In [61]:
scores_svm = cross_val_score(SVC(),x_scaled,y,cv=5)
scores_svm

array([0.85      , 0.86111111, 0.83333333, 0.82222222, 0.77653631])

In [62]:
scores_svm.mean()

0.8286405959031657

In [63]:
model_bagging = BaggingClassifier(
    estimator=SVC(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)

In [64]:
model_bagging.fit(x_train,y_train)

BaggingClassifier(estimator=SVC(), max_samples=0.8, n_estimators=100,
                  oob_score=True, random_state=0)

In [65]:
model_bagging.oob_score_

0.8845618915159944

In [66]:
model_bagging.score(x_test,y_test)

0.8555555555555555

In [67]:
scores_bagging_svm = cross_val_score(
    estimator=model_bagging,
    X=x_scaled,
    y=y,
    cv=5
)
scores_bagging_svm

array([0.85      , 0.86666667, 0.82222222, 0.82777778, 0.80446927])

In [68]:
scores_bagging_svm.mean()

0.8342271880819366

In [70]:
model_dt = DecisionTreeClassifier()
model_dt.fit(x_train,y_train)

DecisionTreeClassifier()

In [71]:
model_dt.score(x_test,y_test)

0.7722222222222223

In [72]:
scores_dt = cross_val_score(
    estimator=DecisionTreeClassifier(),
    X=x_scaled,
    y=y,
    cv=5
)
scores_dt

array([0.79444444, 0.76666667, 0.77222222, 0.72222222, 0.72067039])

In [73]:
scores_dt.mean()

0.7552451893234016

In [74]:
model_bagging = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)

In [75]:
model_bagging.fit(x_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.8,
                  n_estimators=100, oob_score=True, random_state=0)

In [76]:
model_bagging.oob_score_

0.8539638386648123

In [77]:
model_bagging.score(x_test,y_test)

0.8555555555555555

In [81]:
scores_bagging_dt = cross_val_score(
    estimator=model_bagging,
    X=x_scaled,
    y= y,
    cv=5
)
scores_bagging_dt

array([0.9       , 0.82222222, 0.82777778, 0.82222222, 0.74860335])

In [82]:
scores_bagging_dt.mean()

0.8241651148355057